In [1]:
import sys
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd

In [3]:
# df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/NonPromoterSequence.txt', sep = '>', )
# df.dropna(subset=['Unnamed: 0'], how='all', inplace=True)
# df.reset_index(inplace = True)
# df.drop(['EP 1 (+) mt:CoI_1; range -400 to -100.', 'index'], axis = 1, inplace=True) #data cleaning after error found
# df.rename(columns={'Unnamed: 0': "sequence"}, inplace = True)
# df['label'] = 0

# df2 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/PromoterSequence.txt', sep = '>', )
# df2.dropna(subset=['Unnamed: 0'], how='all', inplace=True)
# df2.reset_index(inplace = True)
# df2.drop(['EP 1 (+) mt:CoI_1; range -100 to 200.', 'index'], axis = 1, inplace=True)
# df2.rename(columns={'Unnamed: 0': "sequence"}, inplace = True)
# df2['label'] = 1

# df = pd.concat([df, df2], axis = 0 )

In [4]:
# for seq in df['sequence']:
#     if 'N' in seq:
#         display(df.loc[df['sequence'] == seq])

# df.drop([1822], inplace = True)

In [5]:

# sequence = list(df.loc[:, 'sequence'])
# encoded_seq = []

# def seq2kmer(seq, k):
#     """
#     Convert original sequence to kmers
    
#     Arguments:
#     seq -- str, original sequence.
#     k -- int, kmer of length k specified.
    
#     Returns:
#     kmers -- str, kmers separated by space
#     """
#     kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
#     kmers = " ".join(kmer)
#     return kmers

# for seq in sequence:
#   x = seq2kmer(seq, 6)
#   encoded_seq.append(x)

# print(encoded_seq[0])


In [6]:
# label = list(df.loc[:,'label'])
# new_df = pd.DataFrame(encoded_seq, columns = ['sequence'])

# new_df.insert(1, 'label', label)

# new_df.head()

In [7]:
# new_df.to_csv("dnabert_input.csv", mode='a', header=False)

In [8]:
!pip install transformers

import numpy as np
import os

import joblib
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras import utils
import torch


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 33.4 MB/s 
     |████████████████████████████████| 182 kB 77.0 MB/s 
     |████████████████████████████████| 7.6 MB 47.3 MB/s 


In [9]:
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/dnabert_input.csv', sep = ',', )
df.columns = ['id', 'sequence', 'label']
df.drop(columns = ['id'])


,sequence,label
0,ATTTTT TTTTTA TTTTAC TTTACA TTACAA TACAAG ACAA...,0
1,AGAGAT GAGATA AGATAG GATAGG ATAGGT TAGGTG AGGT...,0
2,TATGTA ATGTAT TGTATA GTATAT TATATA ATATAG TATA...,0
3,AGAAAT GAAATA AAATAA AATAAT ATAATA TAATAG AATA...,0
4,CTTTAA TTTAAA TTAAAA TAAAAT AAAATT AAATTA AATT...,0
...,...,...
22592,CGACAA GACAAA ACAAAG CAAAGT AAAGTT AAGTTT AGTT...,1
22593,CATATC ATATCT TATCTA ATCTAC TCTACA CTACAT TACA...,1
22594,ATACCG TACCGC ACCGCG CCGCGG CGCGGA GCGGAA CGGA...,1
22595,ATTATT TTATTC TATTCC ATTCCG TTCCGA TCCGAA CCGA...,1


In [10]:
df.shape

(22597, 3)

In [11]:
# Import the necessary 
!pip install --upgrade transformers

import transformers
from transformers import AutoModel, AutoTokenizer
from sklearn.model_selection import train_test_split
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss
from tqdm import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
tokenizer = AutoTokenizer.from_pretrained("armheb/DNA_bert_6")

# Load your dataset

data = df

# data['sequence'] = input_sequences

# data.tail()

# Divide the dataset into train, validation, and test sets
train_data, val_data = train_test_split(
    data, 
    test_size=0.1, 
    random_state=42,
    stratify=data['label']
)
val_data, test_data = train_test_split(
    val_data, 
    test_size=0.5, 
    random_state=42,
    stratify=val_data['label']
)

device = torch.device('cuda')
use_cuda = True

train_input = tokenizer.batch_encode_plus(train_data["sequence"].tolist(), 
                                          add_special_tokens=True, 
                                          max_length=512, 
                                          pad_to_max_length=True, 
                                          return_attention_mask=True, 
                                          return_tensors = 'pt')

val_input = tokenizer.batch_encode_plus(val_data["sequence"].tolist(),
                                        add_special_tokens=True, 
                                        max_length=512, 
                                        pad_to_max_length=True, 
                                        return_attention_mask=True, 
                                        return_tensors = 'pt')

test_input = tokenizer.batch_encode_plus(test_data["sequence"].tolist(),
                                         add_special_tokens=True, 
                                         max_length=512, 
                                         pad_to_max_length=True, 
                                         return_attention_mask=True, 
                                         return_tensors = 'pt')

train_input_ids = train_input['input_ids']
train_att_masks = train_input['attention_mask']
train_labels = torch.tensor(train_data['label'].values)

val_input_ids = val_input['input_ids']
val_att_masks = val_input['attention_mask']
val_labels = torch.tensor(val_data['label'].values)

test_input_ids = test_input['input_ids']
test_att_masks = test_input['attention_mask']
test_labels = torch.tensor(test_data['label'].values)

Downloading:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.7k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [13]:
class DNABERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 # 기존 -> num_classes = 2
                 num_classes = 2, # softmax 사용(다중분류)
                 dr_rate=None,
                 params=None):
        super().__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def forward(self, token_ids, attention_mask):
        _, pooler = self.bert(input_ids = token_ids,
                              attention_mask = attention_mask,
                              return_dict=False)
        return self.classifier(pooler) 

In [14]:
bert = AutoModel.from_pretrained("armheb/DNA_bert_6")

model = DNABERTClassifier(bert)

if use_cuda and torch.cuda.is_available():
    model.cuda()

# Define the optimizer, loss function, and number of training epochs
optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)
loss_fn = torch.nn.CrossEntropyLoss(reduction='sum') 


Downloading:   0%|          | 0.00/359M [00:00<?, ?B/s]

Some weights of the model checkpoint at armheb/DNA_bert_6 were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
def metric_batch(output, target):
    pred = output.argmax(1, keepdim=True)
    corrects = pred.eq(target.view_as(pred)).sum().item()
    return corrects


# function to calculate loss per mini-batch
def loss_batch(loss_func, output, target, opt=None):
    loss = loss_func(output, target)
    metric_b = metric_batch(output, target)

    if opt is not None:
        opt.zero_grad()
        loss.backward()
        opt.step()

    return loss.item(), metric_b

In [17]:
loss_history = {"train": [], "val":[]}
batch_history = {'loss': [], 'accuracy': []}
accuracy_history = {"train": [], "val":[]}

train_input_size = len(train_input_ids)
val_input_size = len(val_input_ids)
epochs = 5              # (5, 16) need 3 hour in Colab Free
batch_size = 16         #if your gpu big enough, use 32

for epoch in range(epochs):
    print(f'Epoch {epoch+1}/{epochs}')
    # Set the model to training mode
    model.train()

    epoch_loss = 0.0
    epoch_metric = 0.0

    # Loop over each batch of training data
    for i in tqdm(range(0, train_input_size, batch_size)):
        # Get the current batch of data
        batch_input_ids = train_input_ids[i:i+batch_size].to(device)
        batch_labels = train_labels[i:i+batch_size].to(device)
        batch_att_masks = train_att_masks[i:i+batch_size].to(device)
        # Forward pass
        outputs = model(batch_input_ids, batch_att_masks)

        # Compute the loss
        loss_b, metric_b = loss_batch(loss_fn, outputs, batch_labels, optimizer)

        epoch_loss += loss_b
        if metric_b is not None:
            epoch_metric += metric_b

        if i % 400 == 0:
            print(f'batch {i / batch_size}, loss : {(loss_b/ batch_size):.4f}, accuracy : {(metric_b / batch_size):.4f}')
            batch_history['loss'].append(loss_b / batch_size)
            batch_history['accuracy'].append(metric_b / batch_size)

    epoch_loss = epoch_loss / train_input_size
    epoch_metric = epoch_metric / train_input_size

    loss_history['train'].append(epoch_loss)
    accuracy_history['train'].append(epoch_metric)
    # Set the model to evaluation mode
    model.eval()

    # Evaluate the model on the validation set
    with torch.no_grad():
        val_loss = 0.0
        val_metric = 0.0
        for i in tqdm(range(0, val_input_size, batch_size)):
            # Forward pass
            batch_input_ids = val_input_ids[i:i+batch_size].to(device)
            batch_labels = val_labels[i:i+batch_size].to(device)
            batch_att_masks = val_att_masks[i:i+batch_size].to(device)
            outputs = model(batch_input_ids, batch_att_masks)

            # Compute the loss
            loss_b, metric_b = loss_batch(loss_fn, outputs, batch_labels)
            val_loss += loss_b
            if metric_b is not None:
                val_metric += metric_b

        val_loss = val_loss / val_input_size
        val_metric = val_metric / val_input_size
        loss_history['val'].append(val_loss)
        accuracy_history['val'].append(val_metric)
        # Print the validation loss
    
        print(f'train loss {epoch_loss:.4f}, validation loss {val_loss:.4f}')
        print(f'train accuracy {epoch_metric:.4f}, validation accuracy {val_metric:.4f}')
        print('-'*30)

# Save the finetuned model
# torch.save(model.state_dict(), 'finetuned_model.pth')

Epoch 1/5


  0%|          | 1/1272 [00:01<30:34,  1.44s/it]

batch 0.0, loss : 0.0004, accuracy : 0.0007


  2%|▏         | 26/1272 [00:37<29:42,  1.43s/it]

batch 25.0, loss : 0.0104, accuracy : 0.0156


...

batch 675.0, loss : 0.1259, accuracy : 0.4818


 55%|█████▌    | 701/1272 [16:38<13:31,  1.42s/it]

batch 700.0, loss : 0.1298, accuracy : 0.4999


 56%|█████▋    | 716/1272 [16:59<13:12,  1.42s/it]


KeyboardInterrupt: ignored

In [ ]:
from torchvision import utils
import matplotlib.pyplot as plt

plt.title("Train-Val Loss")
plt.plot(range(1,epochs+1),loss_history["train"], label='train')
plt.plot(range(1,epochs+1),loss_history["val"], label='train')
plt.ylabel("Loss")
plt.xlabel("Training Epochs")
plt.legend()
plt.show()


In [ ]:
print(batch_history)
print(loss_history)
print(accuracy_history)

In [ ]:
# torch.save(model.state_dict(), 'finetuned_model.pth')